# Практическое задание 1: Линейная регрессия, метод градиентного спуска

## №1 Самостоятельно реализовать функцию gradient_descent(X, y), которая по заданной обучающей выборке обучает модель линейной регрессии, оптимизируя функционал методом градиентного спуска (Batch Gradient Descent, GD) и возвращая вектор весов w. В качестве функционала можно выбрать, например, функцию ошибок MSE + -регуляризатор. Использовать матрично-векторные операции для вычисления градиента

In [ ]:
# Убираем предупреждения FutureWarning

import warnings
warnings.filterwarnings('ignore')

import numpy as np

def gradient_descent(X, y, alpha = 0.01, iterations = 1000):
    w = np.zeros(X.shape[1])
    n = X.shape[0]
    for i in range(iterations):
        y_pred = np.dot(X, w)
        for j in range(X.shape[1]):
            w[j] -= alpha * (1/n * 2 * np.sum(X[:, j] * (y_pred - y)))
    return w

## №2 Найти данные, на которых интересно будет решать задачу регрессии. Зависимость целового признака от нецелевых должна быть не слишком сложной, чтобы обученная линейная модель смогла показать приемлимый результат.

Выбран датасет red-wine-quality.csv(https://www.kaggle.com/uciml/red-wine-quality-cortez-et-al-2009), в котором содержится информация о качестве красного вина.

Первые 11 признаков - химические свойства вина, а 12 - оценка качества вина по шкале от 0 до 10.

## №3 Считать данные, выполнить первичный анализ данных, при необходимости произвести чистку данных (Data Cleaning)

In [ ]:
import pandas as pd
data = pd.read_csv('red-wine-quality.csv', sep=',')

data.head()

Данные:
1. fixed acidity - фиксированная кислотность
2. volatile acidity - летучая кислотность
3. citric acid - лимонная кислота
4. residual sugar - остаточный сахар
5. chlorides - хлориды
6. free sulfur dioxide - свободный диоксид серы
7. total sulfur dioxide - общий диоксид серы
8. density - плотность
9. pH - pH
10. sulphates - сульфаты
11. alcohol - алкоголь
12. quality - качество (целевой признак)

Все признаки являются количественными.

In [ ]:
data.info()

In [ ]:
data.describe()

In [ ]:
'''Количество пропусков в данных'''
data.isnull().sum()

Пропусков в данных нет

## №4 Выполнить разведочный анализ (EDA), использовать визуализацию, сделать выводы, которые могут быть полезны при дальнейшем решении задачи регрессии

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

'''Корреляция признаков'''
corr = data.corr()
plt.figure(figsize=(10, 10))
sns.heatmap(corr, vmax=1, square=True, annot=True, cmap='RdYlGn')
plt.title('Correlation between different wine fearures')



Выводы:
- Сильную корреляцию с целевым признаком имеют признаки: alcohol, volatile acidity, citric acid, sulphates
- Слабую корреляцию с целевым признаком имеют признаки: fixed acidity, residual sugar, chlorides, free sulfur dioxide, total sulfur dioxide, density, pH

In [ ]:
plt.figure(figsize=(10, 10))
sns.countplot(x='quality', data=data)
plt.title('Count of quality ranks')

- Больше всего вина с уровнем качества 5 и 6.

In [ ]:
plt.figure(figsize=(10, 10))
sns.boxplot(x='quality', y='alcohol', data=data)
plt.title('Alcohol content in wine with relation to quality')

Выводы:
- Алкоголь в вине влияет на его качество: чем больше содержание алкоголя в вине, тем выше его качество. 
- Есть выбросы в данных, но они не сильно влияют на итоговый результат.

## №5 При необходимости выполнить полезные преобразования данных (например, трансформировать категариальные признаки в количественные), убрать ненужные признаки, создать новые (Feature Engineering)
Необходимости не возникло :)

##  Дополнительное задание. Перед обучением моделей подобрать наилучшее количество (и само подмножество) признаков, например используя Recursive Feature Elimination (RFE)

In [ ]:
'''RFE'''
from sklearn.feature_selection import RFE

X = data.drop(['quality'], axis=1)
y = data['quality']

from sklearn.linear_model import LinearRegression
model = LinearRegression()
rfe = RFE(model, n_features_to_select=6)
fit = rfe.fit(X, y)

for (i, j) in zip(X.columns, fit.support_):
    if j != True:
        X = X.drop([i], axis=1)

list(X.columns.values)

## №6 Случайным образом разбить данные на обучающую и тестовую выборки, используя методы существующих библиотек.

In [ ]:
from sklearn.model_selection import train_test_split
X = data.drop(['quality'], axis=1)
y = data['quality']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## №7 При обучении моделей использовать масштабирование данных

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## №8 Обучить модель на обучающей выборке, используя функцию gradient_descent(X, y). Оценить качество модели на обучающей и тестовой выборках, используя MSE, RMSE и R2. 

In [ ]:
X_train_scaled = np.hstack([np.ones((X_train_scaled.shape[0], 1)), X_train_scaled])
X_test_scaled = np.hstack([np.ones((X_test_scaled.shape[0], 1)), X_test_scaled])
w = gradient_descent(X_train_scaled, y_train)
y_pred = X_test_scaled.dot(w)

In [ ]:
''' Display Mse, RMSE and R2 metrics'''
from sklearn.metrics import mean_squared_error, r2_score
my_gradient_descent_mse = mean_squared_error(y_test, y_pred)
my_gradient_descent_rmse = np.sqrt(my_gradient_descent_mse)
my_gradient_descent_r2 = r2_score(y_test, y_pred)
print('MSE: ', my_gradient_descent_mse)
print('RMSE: ', my_gradient_descent_rmse)
print('R2: ', my_gradient_descent_r2)


## №9 Обучить модель, используя существующую библиотеку. Например, в sklearn для -регуляризатора можно использовать Ridge. Сравнить качество с вашей реализацией.

In [ ]:
from sklearn.linear_model import Ridge
model = Ridge()
model.fit(X_train_scaled, y_train)
y_pred = model.predict(X_test_scaled)


In [ ]:
''' Display Mse, RMSE and R2 metrics'''
Ridge_mse = mean_squared_error(y_test, y_pred)
Ridge_rmse = np.sqrt(Ridge_mse)
Ridge_r2 = r2_score(y_test, y_pred)

print('MSE: ', Ridge_mse)
print('RMSE: ', Ridge_rmse)
print('R2: ', Ridge_r2)


In [ ]:
from prettytable import PrettyTable
table = PrettyTable()
table.field_names = ['Model', 'MSE', 'RMSE', 'R2']
table.add_row(['Ridge', Ridge_mse, Ridge_rmse, Ridge_r2])
table.add_row(['Gradient_descent', my_gradient_descent_mse, my_gradient_descent_rmse, my_gradient_descent_r2])

print(table)


Выводы: самостоятельная и библиотечная реализации дают очень схожие результаты

## №10 Повторить тоже самое, но используя кросс-валидацию

In [ ]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(model, X_train_scaled, y_train, cv=5, scoring='neg_mean_squared_error')
print('MSE: ', scores.mean())
print('RMSE: ', np.sqrt(-scores.mean()))
print('R2: ', r2_score(y_test, y_pred))

## №11 Создать таблицу, со строками (mse-train, mse-test, rmse-train, rmse-test, r2-train, r2-test) и столбцами (Fold1, Fold2, ..., Foldk, E, STD), где k --- количество фолдов в кросс-валидации, E --- мат. ожидание и STD --- стандартное отклонение. Сделать выводы.

In [ ]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate

rdg_model = Ridge()
scoring = {'neg_mse': 'neg_mean_squared_error',
           'neg_rmse': 'neg_root_mean_squared_error',
           'r2': 'r2'}

kf = KFold(n_splits = 5, random_state=42, shuffle=True)
scores = cross_validate(rdg_model, X, y, cv = kf,
                                scoring=scoring,
                                return_train_score = True)

summary_table = pd.DataFrame(scores)

summary_table.drop(['fit_time', 'score_time'], axis = 1, inplace = True)
summary_table['test_mse'] = -1*summary_table['test_neg_mse']
summary_table['train_mse'] = -1*summary_table['train_neg_mse']
summary_table['test_rmse'] = -1*summary_table['test_neg_rmse']
summary_table['train_rmse'] = -1*summary_table['train_neg_rmse']
summary_table.drop(['test_neg_mse', 'train_neg_mse', 'test_neg_rmse', 'train_neg_rmse'], axis = 1, inplace = True)
summary_table = summary_table.transpose()
summary_table.set_axis(['Fold2','Fold3','Fold4','Fold5','Fold6'], axis=1, copy=False)
summary_table['E'] = summary_table.mean(axis = 1)
summary_table['STD'] = summary_table.std(axis = 1)

summary_table.style.format("{:.5f}")

Выводы:
- Модель с кросс-валидацией работает лучше, чем без нее
- Метрики качества модели на тестовой выборке ниже, чем на обучающей, что говорит о том, что модель не переобучена
- При увеличении количества фолдов качество модели улучшается






## Доп. задание 2. Также самостоятельно реализовать метод стохастического градиентного спуска (Stochastic Gradient Descent, SGD), обучить модели и добавить их во все сравнения

In [ ]:
'''stochastic gradient descent'''
def stochastic_gradient_descent(X, y, w, eta=0.01, n_iterations=100):
    m = X.shape[0]
    for iteration in range(n_iterations):
        for i in range(m):
            random_index = np.random.randint(m)
            xi = X[random_index:random_index+1]
            yi = y[random_index:random_index+1]
            gradients = 2/m * xi.T.dot(xi.dot(w) - yi)
            w = w - eta * gradients
    return w

In [ ]:
w = np.zeros(X_train_scaled.shape[1])
w = stochastic_gradient_descent(X_train_scaled, y_train, w)
y_pred = X_test_scaled.dot(w)

''' Display Mse, RMSE and R2 metrics'''
my_stochastic_gradient_descent_mse = mean_squared_error(y_test, y_pred)
my_stochastic_gradient_descent_rmse = np.sqrt(my_stochastic_gradient_descent_mse)
my_stochastic_gradient_descent_r2 = r2_score(y_test, y_pred)

print('MSE: ', my_stochastic_gradient_descent_mse)
print('RMSE: ', my_stochastic_gradient_descent_rmse)
print('R2: ', my_stochastic_gradient_descent_r2)

## Доп. задание 3. Также самостоятельно реализовать метод мини-пакетного градиентного спуска (Mini Batch Gradient Descent), обучить модели и добавить их во все сравнения.

In [ ]:
def mini_batch_gradient_descent(X, y, batch_size=20, alpha=0.01, n_iterations=100):
    w = np.zeros(X.shape[1])
    m = X.shape[0]
    for i in range(n_iterations):
        for j in range(0, m, batch_size):
            X_batch = X[j:j+batch_size]
            y_batch = y[j:j+batch_size]
            w = w - alpha * (1/batch_size) * np.dot(X_batch.T, (np.dot(X_batch, w) - y_batch))
    return w

In [ ]:
w = mini_batch_gradient_descent(X_train_scaled, y_train)
y_pred = X_test_scaled.dot(w)

''' Display Mse, RMSE and R2 metrics'''
my_mini_batch_gradient_descent_mse = mean_squared_error(y_test, y_pred)
my_mini_batch_gradient_descent_rmse = np.sqrt(my_mini_batch_gradient_descent_mse)
my_mini_batch_gradient_descent_r2 = r2_score(y_test, y_pred)

print('MSE: ', my_mini_batch_gradient_descent_mse)
print('RMSE: ', my_mini_batch_gradient_descent_rmse)
print('R2: ', my_mini_batch_gradient_descent_r2)

### Свобдная таблица с результатами

In [ ]:
from prettytable import PrettyTable
x = PrettyTable()
x.field_names = ["Model", "MSE", "RMSE", "R2"]
x.add_row(["My Linear Regression", my_gradient_descent_mse, my_gradient_descent_rmse, my_gradient_descent_r2])
x.add_row(["Ridge Regression", Ridge_mse, Ridge_rmse, Ridge_r2])
x.add_row(["My Stochastic Gradient Descent", my_stochastic_gradient_descent_mse, my_stochastic_gradient_descent_rmse, my_stochastic_gradient_descent_r2])
x.add_row(["My Mini Batch Gradient Descent", my_mini_batch_gradient_descent_mse, my_mini_batch_gradient_descent_rmse, my_mini_batch_gradient_descent_r2])
print(x)